In [6]:
import sys
import os

sys.path.insert(0, "/home/pavithra/projects/Learnings/")
from open_ai_keys import LONGSMITH_LEARNING_API_KEY, GEMINI_LEARNING_API_KEY
# Set the Key as an environment variable.
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = LONGSMITH_LEARNING_API_KEY

# Setting up the API key for the gemini model.
os.environ["GOOGLE_API_KEY"] = GEMINI_LEARNING_API_KEY

In [ ]:
!pip install -qU langchain-google-genai

: 

In [ ]:
ninjaac-cjrkbk
